# Getting going with MagIC and rockmagpy

## Background on MagIC

The Magnetics Information Consortium (MagIC), hosted at http://earthref.org/MagIC is a database that serves as a Findable, Accessible, Interoperable, Reusable (FAIR) archive for paleomagnetic and rock magnetic data. Its data model is fully described here: [https://www2.earthref.org/MagIC/data-models/3.0](https://www2.earthref.org/MagIC/data-models/3.0). Each contribution is associated with a publication via the DOI.  

There are nine data tables. The fields associated with these tables are detailed here: https://www2.earthref.org/MagIC/data-models/3.0

- contribution: metadata of the associated publication.
- locations: metadata for locations, which are groups of sites (e.g., stratigraphic section, region, etc.)
- sites: metadata and derived data at the site level (units with a common expectation)
- samples: metadata and derived data at the sample level.
- specimens: metadata and derived data at the specimen level.
- measurements: metadata and measurements at the specimen level.
- criteria: criteria by which data are deemed acceptable
- ages: ages and metadata for sites/samples/specimens
- images: associated images and plots.

## Importing data from MagIC

This notebook illustrates the two approaches that can be taken to import data from MagIC into a Jupyter notebook:
- **From a locally hosted (i.e. downloaded) MagIC contribution**
- **Directly from the MagIC database (using the API)**
  - either from a public contribution or
  - from a private contribution

The MagIC file is then unpacked into its constituent files and loaded as a contribution object. The notebook then summarizes the specimens and the experiments that have been run on them within the data. Finally, it is illustrated how data from a single specimen and single experiment on that specimen can be isolated.

In the other rockmagpy notebooks, these import methods are used to bring in contribution data. The filtering methods to access data from specific specimens and experiments are used either in the notebooks or within the `rockmag.py` functions.

## Import scientific python libraries

Run the cell below to import the functions needed for the notebook including `rockmagpy` which is imported as `import pmagpy.rockmag as rmag`.

In [ ]:
import pmagpy.ipmag as ipmag
import pmagpy.rockmag as rmag
import pmagpy.contribution_builder as cb
import pandas as pd
pd.set_option('display.max_columns', 100)
from bokeh.io import output_notebook
output_notebook(hide_banner=True)

## Unpack data from a local MagIC file

Within the folder `./example_data/ECMB`, there is a MagIC contribution called `'magic_contribution_20213.txt'` that has been downloaded already from the MagIC database and is associated with this publication:

> Swanson-Hysell, N.L., Avery, M.S., Zhang, Y., Hodgin, E.B., Sherwood, R.J., Apen, F.E., Boerboom, T.J., Keller, C.B., and Cottle, J.M. (2021), The paleogeography of Laurentia in its early years: new constraints from the Paleoproterozoic East Central Minnesota batholith, Tectonics, doi:10.1130/G47873.1.

 Running the code cell below unpacks that MagIC contribution so that we can visualize and analyze the measurement data. We can then create a contribution object that has all of the MagIC tables. This approach can be applied to MagIC contributions that are downloaded from the database.

In [ ]:
# set the directory path (dir_path) to the data
dir_path = './example_data/ECMB'

# set the name of the MagIC file
ipmag.unpack_magic('magic_contribution_20213.txt', 
                     dir_path = dir_path,
                     input_dir_path = dir_path,
                     print_progress=False)

ECMB_contribution = cb.Contribution(dir_path)

### Inspect the measurements table

Within the MagIC contribution object, there is a measurements table that we can inspect.

In [ ]:
ECMB_measurements = ECMB_contribution.tables['measurements'].df
ECMB_measurements.dropna(axis=1, how='all').tail(2) # see the last 2 measurements without empty columns

### List out the experiments

We can use the function `rmag.make_experiment_df()` to list out the experiments that are a part of this MagIC contribution. We will exclude the experiments that are associated with AF and thermal demagnetization to obtain directional magnetization data in order to focus on the hysteresis experiments and low-temperature experiments.

The method codes associated with the experiments are from the controlled vocabularies in the MagIC database https://www2.earthref.org/MagIC/method-codes.

| Method Code    | Definition |
| -------------- | ---------- |
| LP-BCR-BF      | Coercivity of remanence: Back field method | 
| LP-HYS | Hysteresis loops |
| LP-CW-SIRM:LP-MC| Cycling between cooling and warming: Room temperature SIRM; Measured while cooling |
| LP-CW-SIRM:LP-MW| Cycling between cooling and warming: Room temperature SIRM; Measured while warming |
| LP-FC          | Field cooled: Remanent magnetization measured on warming |
| LP-ZFC         | Zero field cooled: Remanent magnetization measured on warming |

In [ ]:
ECMB_experiments = rmag.make_experiment_df(ECMB_measurements,
                                           exclude_method_codes=['LP-DIR-AF','LP-DIR-T'])
ECMB_experiments

### Analyze and plot a hysteresis loop

In [ ]:
#specify the experiment name and specimen name
experiment_name = 'IRM-VSM3-LP-HYS-218845'
specimen_name = 'NED1-5c'

experiment_hyst = rmag.experiment_selection(ECMB_measurements,
                                            experiment_name)
experiment_results = rmag.process_hyst_loop(experiment_hyst['meas_field_dc'].values, 
                                            experiment_hyst['magn_mass'].values, specimen_name)

### Plot low-temperature remanence experiments

In [ ]:
specimen_name = 'NED4-1c'
fc_data, zfc_data, rtsirm_cool_data, rtsirm_warm_data = rmag.extract_mpms_data_dc(ECMB_measurements, specimen_name)
rmag.plot_mpms_dc(fc_data, zfc_data, rtsirm_cool_data, rtsirm_warm_data, 
                  interactive=True, plot_derivative=True)

## Direct download of a published MagIC contribution

We can directly download a public MagIC contribution by providing the MagIC ID to the function `ipmag.download_magic_from_id()`.

Let's do that and pull in Rock Magnetic Bestiary data associated with oxyhydroxides that have been published in the MagIC database here: https://earthref.org/MagIC/20427 with ```'20427'``` being the MagIC ID.

In [ ]:
# set the MagIC ID for the data set here
magic_id = '20427'

# set where you want the downloaded data to go
dir_path = 'example_data/RMB_oxyhydroxides'

result, magic_file = ipmag.download_magic_from_id(magic_id, directory=dir_path)
ipmag.unpack_magic(magic_file, dir_path, print_progress=False)
oxy_contribution = cb.Contribution(dir_path)
oxy_measurements = oxy_contribution.tables['measurements'].df

### Examine experiments

In [ ]:
oxy_experiments = rmag.make_experiment_df(oxy_measurements)
oxy_experiments

### Plot low-temperature data

Input one of the specimens with DC low-temperature remanence experiments in order to plot the data. Enter one of the following in the specimen_name below ferroxyhyte_Princeton-1985-M2-1, goethite_AA 19496-1 , goethite_Pfizer-YLO1888D-1, goethite_Pfizer-YO-5087-1,goethite_Pfizer-YO-6087-1, lepidocrocite_Pfizer_1

In [ ]:
specimen_name = 'goethite_Pfizer-YO-5087-1'
fc_data, zfc_data, rtsirm_cool_data, rtsirm_warm_data = rmag.extract_mpms_data_dc(oxy_measurements, specimen_name)
rmag.plot_mpms_dc(fc_data, zfc_data, rtsirm_cool_data, rtsirm_warm_data, 
                  interactive=True, drop_first=True)

## Let's upload a contribution! 

1. Navigate to download this MagIC file which contains data generated by one of the IRM Summer School groups in 2024: https://github.com/Institute-for-Rock-Magnetism/2025_rockmagpy_workshop/blob/main/2_MagIC_hysteresis/example_data/SSRM_example/example_MagIC_file.txt

2. Download the MagIC file locally to your computer by clicking the download button:

   <img src="images/download_example.png" width="500"/>

3. Log-in to MagIC/Earthref (https://www2.earthref.org/MagIC).

4. Go to upload tool (https://www2.earthref.org/MagIC/upload)

5. Drag and drop the file into the upload tool.

   <img src="images/upload_data.png" width="500"/>

6. Upon uploading, we can examine the tables which are parsed and matched up with the table name and data model fields.

   <img src="images/import_data.png" width="500"/>

7. Click the upload button in the lower right, which will bring you to an upload page where you can click through to upload data into your private workspace.
   
   <img src="images/private_workspace.png" width="500"/>

8. Add the lab for the study `Institute for Rock Magnetism` and then click save.
   
   <img src="images/manage_contribution.png" width="500"/>

9. These data are now in MagIC as a private contribution. We could click validate to validate the contribution and then publish it, but we won't do that with these data. What we can do is to get a private link that can be shared. Sharing such a link enables data to be made available for peer review. Note that the DOI for the contribution (Future Data DOI) is already available at this stage which enables the data to be properly referenced within a manuscript when it is submitted. 

   <img src="images/private_share.png" width="400"/>

10. We can also bring data from a private in our workspace into a notebook for analysis which we will do below.

### Download and unpack data directly from this private MagIC contribution

To bring in a data from a contribution that has been uploaded to MagIC, but that is not yet publically published, you need to both set the `magic_id` and the `share_key` for the data MagIC contribution which you can find by clicking on the "Share" button in the MagIC database. Here we will do that with data developed by participants in one of the groups from the IRM Summer School that we just uploaded

<figure>
  <img src="https://raw.githubusercontent.com/PmagPy/RockmagPy-notebooks/main/book/images/MagIC_private_contribution_1.png" alt="Description of the image" width="800">
</figure>

Copy the share key which is the code after the last backslash to the `share_key` parameter in the cell below.

<figure>
  <img src="https://raw.githubusercontent.com/PmagPy/RockmagPy-notebooks/main/book/images/MagIC_private_contribution_2.png" alt="Description of the image" width="800">
</figure>

In [ ]:
# set the MagIC ID for the data set here
magic_id = ''

# provide the share key for the data set
share_key = ''

# set where you want the downloaded data to go
dir_path = 'example_data/SSRM_example'

result, magic_file = ipmag.download_magic_from_id(magic_id, directory=dir_path, share_key=share_key)
ipmag.unpack_magic(magic_file, dir_path, print_progress=False)
SSRM_contribution = cb.Contribution(dir_path)

### List out the experiments 

In [ ]:
SSRM_experiments = rmag.make_experiment_df(SSRM_measurements)
SSRM_experiments

### Plot a thermomagnetic curve

In the cell below, put in an experiment name associated with one of the LP-X-T experiments. 

In [ ]:
experiment_name = ''
selected_experiment = rmag.experiment_selection(SSRM_measurements,
                                                experiment_name)

rmag.plot_X_T(selected_experiment, 
              temp_unit='C', 
              remove_holder=True, 
              plot_derivative=True,
              plot_inverse=True,
              interactive=True,
              figsize=(8,3))

### Plot a backfield curve

In the cell below, put in an experiment name associated with one of the backfield curve (LP-BCR-BF) experiments. The data for specimen MA1-OX-r (experiment IRM-VSM4-LP-BCR-BF-239379 is particularly interesting).

In [ ]:
experiment_name = ''
selected_experiment = rmag.experiment_selection(SSRM_measurements,
                                                experiment_name)
processed_experiment, Bcr = rmag.backfield_data_processing(selected_experiment,
                                                           smooth_mode='spline',
                                                           smooth_frac=0.0001)
rmag.plot_backfield_data(processed_experiment, Bcr=Bcr, interactive=True)

### Plot a hysteresis loop

In [ ]:
#specify the experiment name and specimen name
experiment_name = 'IRM-VSM3-LP-HYS-239371'
specimen_name = 'MA1-OX-r'

hyst_experiment = rmag.experiment_selection(SSRM_measurements,
                                            experiment_name)
experiment_results = rmag.process_hyst_loop(hyst_experiment['meas_field_dc'].values, 
                                            hyst_experiment['magn_mass'].values, specimen_name)

### Plot low-temperature remanence experiments

In [ ]:
specimen_name = 'MA1-5-r gelcap'
fc_data, zfc_data, rtsirm_cool_data, rtsirm_warm_data = rmag.extract_mpms_data_dc(SSRM_measurements, specimen_name)
rmag.plot_mpms_dc(fc_data, zfc_data, rtsirm_cool_data, rtsirm_warm_data,
                  plot_derivative=True, interactive=True)

### Make a static plot and save the figure

In [ ]:
MPMS_figure = rmag.plot_mpms_dc(fc_data, zfc_data, rtsirm_cool_data, rtsirm_warm_data, 
                                interactive=False, plot_derivative=True,return_figure=True)

MPMS_figure.savefig('example_data/SSRM_example/MPMS_plot.pdf')

### Customize a static plot

In [ ]:
rmag.plot_mpms_dc(fc_data, zfc_data, rtsirm_cool_data, rtsirm_warm_data, 
                   fc_color='orange', zfc_color='purple', rtsirm_cool_color='blue', rtsirm_warm_color='darkred',
                   fc_marker='^', zfc_marker='o', rtsirm_cool_marker='*', rtsirm_warm_marker='s',
                   symbol_size=4, interactive=False, plot_derivative=True)